## Multi Layer Perceptron for MNIST

### Dataset loading and the model building

In [30]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test   = tf.keras.utils.normalize(x_test, axis=1)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])
model.fit(x=x_train, y=y_train, epochs=3, validation_split=.3)

Train on 42000 samples, validate on 18000 samples
Epoch 1/3
42000/42000 [==============================] - 9s 207us/step - loss: 0.3110 - acc: 0.9094 - val_loss: 0.1790 - val_acc: 0.9462
Epoch 2/3
42000/42000 [==============================] - 8s 193us/step - loss: 0.1266 - acc: 0.9603 - val_loss: 0.1360 - val_acc: 0.9590
Epoch 3/3
10000/10000 [==============================] - 1s 60us/step
('Test loss = ', 0.10736036267727614, 'Test accuracy = ', 0.9654)


### Evaluating the model on the test dataset

In [44]:
test_loss, test_acc = model.evaluate(x=x_test, y=y_test)
print 'Test loss = %8.2f' %test_loss, ' and Test accuracy = %8.2f' %test_acc

10000/10000 [==============================] - 1s 60us/step
Test loss =     0.11  and Test accuracy =     0.97
